# Linha do tempo

In [220]:
import glob
import re
import pandas as pd
from yattag import Doc
import unicodedata
import os
import json
import numpy as np

In [221]:
doc, tag, text = Doc().tagtext()

In [222]:
txt_files = glob.glob('text/**/**/*.txt')
block_files = glob.glob('blocks/**/**/*.json')

In [223]:
# determinando um processo específico
all_acts = []
processo = '00401-00019051/2018-39'
pregao = '001/2020'

In [224]:
for txt in txt_files:
    instances = re.findall(r'((?:(?:AVISO|RESULTADO[s]|EXTRATO|EDITAL).*[\s ])(?:[\s\S]*?)\.)\n',
                            unicodedata.normalize('NFKD', open(txt, encoding='utf-8').read()), 
                            re.IGNORECASE)
    for instance in instances:
        if processo in instance:
            all_acts.append(txt)
            all_acts.append(instance)

In [225]:
df_dict = {
    'descricao_objeto': [],
    'data_dodf': [],
    'texto': [],
}

regex_dict = {
    'descricao_objeto': '(?:(?:aviso|resultado[s]|extrato|edital).*\n)',
    'data_dodf': '\d{2}-\d{2}-\d{4}',
}

In [226]:
for act in range(0, len(all_acts)):
    if act % 2 == 0:
        dodf = all_acts[act]
        match_data = re.search(regex_dict['data_dodf'], str(dodf), re.IGNORECASE)
        if match_data:
            df_dict['data_dodf'].append(match_data.group())
        else:
            df_dict['data_dodf'].append(np.nan)
    else:
        df_dict['texto'].append(all_acts[act])
        match_descricao = re.search(regex_dict['descricao_objeto'], str(all_acts[act]), re.IGNORECASE)

        if match_descricao:
            df_dict['descricao_objeto'].append(match_descricao.group())
        else:
            df_dict['descricao_objeto'].append(np.nan)

df = pd.DataFrame.from_dict(dict([ (k,pd.Series(v,dtype=pd.StringDtype())) for k,v in df_dict.items() ]))
df.to_csv('linha_tempo.csv')
df

,descricao_objeto,data_dodf,texto
0,AVISO DE HOMOLOGACAO E CONVOCACAO,14-02-2019,AVISO DE HOMOLOGACAO E CONVOCACAO PREGAO ELETR...
1,AVISO DE RESULTADO,15-02-2019,AVISO DE RESULTADO PREGAO ELETRONICO No 01/201...
2,EXTRATO DO CONTRATO No 006/2019,22-02-2019,EXTRATO DO CONTRATO No 006/2019 PROCESSO: 0040...
3,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,04-12-2019,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...
4,AVISO DE ABERTURA DE LICITACAO,25-01-2019,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
